# Openclassrooms PJ5 : Online Retail dataset :  modelisation notebook 

In [422]:
%matplotlib inline

#%load_ext autoreload  # Autoreload has a bug : when you modify function in source code and run again, python kernel hangs :(
#%autoreload 2

import datetime as dt

import sys, importlib

from functions import *
importlib.reload(sys.modules['functions'])

import pandas as pd

pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)

import datetime as dt

import os
import zipfile
import urllib

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import qgrid

import glob

from pandas.plotting import scatter_matrix

from sklearn.model_selection import StratifiedShuffleSplit

from sklearn.model_selection import GridSearchCV

DATA_PATH = os.path.join("datasets", "onlineretail")
DATA_PATH = os.path.join(DATA_PATH, "out")

DATA_PATH_FILE_INPUT = os.path.join(DATA_PATH, "OnlineRetail_transformed.csv")


ALL_FEATURES = []

MODEL_FEATURES=['InvoiceNo', 'InvoiceDate', 'CustomerID', 'TotalPrice', 'DescriptionNormalized', 'InvoiceMonth', 'StockCode']

plt.rcParams["figure.figsize"] = [16,9] # Taille par défaut des figures de matplotlib

import seaborn as sns
sns.set()

#import common_functions

####### Paramètres pour sauver et restaurer les modèles :
import pickle
####### Paramètres à changer par l'utilisateur selon son besoin :

RECOMPUTE_GRIDSEARCH = True  # CAUTION : computation is several hours long
SAVE_GRID_RESULTS = False # If True : grid results object will be saved to pickle files that have GRIDSEARCH_FILE_PREFIX
LOAD_GRID_RESULTS = False # If True : grid results object will be loaded from pickle files that have GRIDSEARCH_FILE_PREFIX
                          # Grid search results are loaded with full samples (SAMPLED_DATA must be False)

'''
RECOMPUTE_GRIDSEARCH = True  # CAUTION : computation is several hours long
SAVE_GRID_RESULTS = True # If True : grid results object will be saved to pickle files that have GRIDSEARCH_FILE_PREFIX
LOAD_GRID_RESULTS = False # If True : grid results object will be loaded from pickle files that have GRIDSEARCH_FILE_PREFIX
'''
#GRIDSEARCH_CSV_FILE = 'grid_search_results.csv'

GRIDSEARCH_FILE_PREFIX = 'grid_search_results_'

EXECUTE_INTERMEDIATE_MODELS = True # If True: every intermediate model (which results are manually analyzed in the notebook) will be executed


# Necessary for predictors used in the notebook :
from sklearn.linear_model import LinearRegression
from sklearn.compose import TransformedTargetRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

from sklearn.preprocessing import PolynomialFeatures

### For progress bar :
from tqdm import tqdm_notebook as tqdm

# Statsmodel : 
import statsmodels.formula.api as smf

import statsmodels.api as sm
from scipy import stats

SAVE_API_MODEL = True # If True : API model ill be saved
API_MODEL_PICKLE_FILE = 'API_model_PJ5.pickle'


# Load data

In [423]:
df = load_data(DATA_PATH_FILE_INPUT)

In [424]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 401604 entries, 0 to 401603
Data columns (total 11 columns):
InvoiceNo                401604 non-null object
StockCode                401604 non-null object
Description              401604 non-null object
Quantity                 401604 non-null int64
InvoiceDate              401604 non-null object
UnitPrice                401604 non-null float64
CustomerID               401604 non-null object
Country                  401604 non-null object
TotalPrice               401604 non-null float64
DescriptionNormalized    401604 non-null object
InvoiceMonth             401604 non-null object
dtypes: float64(2), int64(1), object(8)
memory usage: 33.7+ MB


In [425]:
df, df_train, df_test = custom_train_test_split_sample(df, 'TotalPrice')

In [426]:
df_train.reset_index(inplace=True)
df_test.reset_index(inplace=True)

In [427]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 361443 entries, 0 to 361442
Data columns (total 12 columns):
index                    361443 non-null int64
InvoiceNo                361443 non-null object
StockCode                361443 non-null object
Description              361443 non-null object
Quantity                 361443 non-null int64
InvoiceDate              361443 non-null object
UnitPrice                361443 non-null float64
CustomerID               361443 non-null object
Country                  361443 non-null object
TotalPrice               361443 non-null float64
DescriptionNormalized    361443 non-null object
InvoiceMonth             361443 non-null object
dtypes: float64(2), int64(2), object(8)
memory usage: 33.1+ MB


In [428]:
df_train_ori = df_train.copy(deep=True)
df_test_ori = df_test.copy(deep=True)

# Top value products (must be saved with the model, and passed to it)

In [429]:
TOP_VALUE_THRESHOLD = 10
top_value_products = df_gbproduct.sort_values(ascending=False).head(TOP_VALUE_THRESHOLD).index  # Get top value products

In [430]:
df[df['StockCode'].str.isalpha()]['StockCode'].unique()

array(['POST', 'D', 'M', 'PADS', 'DOT', 'CRUK'], dtype=object)

In [436]:
df[df['StockCode'] == 'CRUK']['TotalPrice'].sum()

-7933.43

In [437]:
df[df['StockCode'] == 'D']

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalPrice,DescriptionNormalized,InvoiceMonth
141,C536379,D,Discount,-1,2010-12-01 09:41:00,27.50,14527,United Kingdom,-27.50,Discount,2010-12-01 00:00:00
6596,C537164,D,Discount,-1,2010-12-05 13:21:00,29.29,14527,United Kingdom,-29.29,Discount,2010-12-01 00:00:00
9977,C537597,D,Discount,-1,2010-12-07 12:34:00,281.00,15498,United Kingdom,-281.00,Discount,2010-12-01 00:00:00
12450,C537857,D,Discount,-1,2010-12-08 16:00:00,267.12,17340,United Kingdom,-267.12,Discount,2010-12-01 00:00:00
20274,C538897,D,Discount,-1,2010-12-15 09:14:00,5.76,16422,United Kingdom,-5.76,Discount,2010-12-01 00:00:00
20275,C538897,D,Discount,-1,2010-12-15 09:14:00,42.50,16422,United Kingdom,-42.50,Discount,2010-12-01 00:00:00
20799,C539003,D,Discount,-1,2010-12-15 12:35:00,26.93,14527,United Kingdom,-26.93,Discount,2010-12-01 00:00:00
24885,C539589,D,Discount,-1,2010-12-20 13:14:00,13.88,14527,United Kingdom,-13.88,Discount,2010-12-01 00:00:00
27551,C540171,D,Discount,-1,2011-01-05 12:26:00,22.97,14527,United Kingdom,-22.97,Discount,2011-01-01 00:00:00
55316,C543752,D,Discount,-1,2011-02-11 14:12:00,64.27,14031,United Kingdom,-64.27,Discount,2011-02-01 00:00:00


In [417]:
df_gbproduct[df_gbproduct.index.isin(df[df['StockCode'].str.isalpha()]['StockCode'].unique())]

StockCode
DOT     11906.360
M       53419.930
PADS        0.003
POST    77803.960
Name: TotalPrice, dtype: float64

In [418]:
df[df['StockCode'].isin(['POST', 'D', 'M', 'PADS', 'DOT', 'CRUK'])].shape

(1769, 11)

In [419]:
df.shape

(401604, 11)

In [420]:
df.drop(index=df[df['StockCode'].isin(['POST', 'D', 'M', 'PADS', 'DOT', 'CRUK'])].index, axis=0, inplace=True)

In [421]:
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalPrice,DescriptionNormalized,InvoiceMonth
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30,WHITE HANGING HEART T-LIGHT HOLDER,2010-12-01 00:00:00
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,WHITE METAL LANTERN,2010-12-01 00:00:00
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00,CREAM CUPID HEARTS COAT HANGER,2010-12-01 00:00:00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,KNITTED UNION FLAG HOT WATER BOTTLE,2010-12-01 00:00:00
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,RED WOOLLY HOTTIE WHITE HEART.,2010-12-01 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...
401599,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680,France,10.20,PACK OF 20 SPACEBOY NAPKINS,2011-12-01 00:00:00
401600,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680,France,12.60,CHILDREN'S APRON DOLLY GIRL,2011-12-01 00:00:00
401601,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680,France,16.60,CHILDRENS CUTLERY DOLLY GIRL,2011-12-01 00:00:00
401602,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680,France,16.60,CHILDRENS CUTLERY CIRCUS PARADE,2011-12-01 00:00:00


In [372]:
df[df['StockCode'] == 'DOT']

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalPrice,DescriptionNormalized,InvoiceMonth
228469,564764,DOT,DOTCOM POSTAGE,1,2011-08-30 10:49:00,11.17,14096,United Kingdom,11.17,DOTCOM POSTAGE,2011-08-01 00:00:00
231867,565383,DOT,DOTCOM POSTAGE,1,2011-09-02 15:45:00,16.46,14096,United Kingdom,16.46,DOTCOM POSTAGE,2011-09-01 00:00:00
239480,566217,DOT,DOTCOM POSTAGE,1,2011-09-09 15:17:00,13.16,14096,United Kingdom,13.16,DOTCOM POSTAGE,2011-09-01 00:00:00
244017,566566,DOT,DOTCOM POSTAGE,1,2011-09-13 12:32:00,85.58,14096,United Kingdom,85.58,DOTCOM POSTAGE,2011-09-01 00:00:00
253972,567656,DOT,DOTCOM POSTAGE,1,2011-09-21 14:40:00,878.55,14096,United Kingdom,878.55,DOTCOM POSTAGE,2011-09-01 00:00:00
262581,568346,DOT,DOTCOM POSTAGE,1,2011-09-26 15:28:00,688.08,14096,United Kingdom,688.08,DOTCOM POSTAGE,2011-09-01 00:00:00
271861,569246,DOT,DOTCOM POSTAGE,1,2011-10-03 09:57:00,490.25,14096,United Kingdom,490.25,DOTCOM POSTAGE,2011-10-01 00:00:00
286166,570488,DOT,DOTCOM POSTAGE,1,2011-10-10 17:12:00,635.01,14096,United Kingdom,635.01,DOTCOM POSTAGE,2011-10-01 00:00:00
295803,571441,DOT,DOTCOM POSTAGE,1,2011-10-17 13:31:00,743.62,14096,United Kingdom,743.62,DOTCOM POSTAGE,2011-10-01 00:00:00
307827,572552,DOT,DOTCOM POSTAGE,1,2011-10-24 17:07:00,638.30,14096,United Kingdom,638.30,DOTCOM POSTAGE,2011-10-01 00:00:00


In [348]:
top_value_products

Index(['23843', '22423', '85123A', '85099B', '23166', 'POST', '47566', '84879',
       'M', '23084'],
      dtype='object', name='StockCode')

In [358]:
df_nocancel[df_nocancel['StockCode'] == 'POST']['TotalPrice'].sum()

77803.96

In [360]:
df_gbproduct.sort_values(ascending=False).head(10)

StockCode
23843     168469.60
22423     142264.75
85123A    100547.45
85099B     85040.54
23166      81416.73
POST       77803.96
47566      68785.23
84879      56413.03
M          53419.93
23084      51251.24
Name: TotalPrice, dtype: float64

In [350]:
pd.set_option('display.max_rows', 200)
df[df['StockCode'] == 'POST'].sample(200)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalPrice,DescriptionNormalized,InvoiceMonth
67873,545181,POST,POSTAGE,1,2011-02-28 14:45:00,18.00,12509,France,18.00,POSTAGE,2011-02-01 00:00:00
395772,580965,POST,POSTAGE,3,2011-12-06 14:52:00,15.00,12417,Belgium,45.00,POSTAGE,2011-12-01 00:00:00
350946,576630,POST,POSTAGE,1,2011-11-16 08:38:00,18.00,13816,Germany,18.00,POSTAGE,2011-11-01 00:00:00
329157,574679,POST,POSTAGE,3,2011-11-06 12:38:00,18.00,12645,Germany,54.00,POSTAGE,2011-11-01 00:00:00
149147,555407,POST,POSTAGE,1,2011-06-03 07:48:00,15.00,12775,Netherlands,15.00,POSTAGE,2011-06-01 00:00:00
70801,545533,POST,POSTAGE,2,2011-03-03 14:04:00,18.00,12524,Germany,36.00,POSTAGE,2011-03-01 00:00:00
395433,580955,POST,POSTAGE,1,2011-12-06 14:22:00,28.00,12442,Spain,28.00,POSTAGE,2011-12-01 00:00:00
167944,557754,POST,POSTAGE,1,2011-06-22 13:12:00,700.00,12432,Norway,700.00,POSTAGE,2011-06-01 00:00:00
236992,565930,POST,POSTAGE,5,2011-09-08 10:32:00,18.00,12685,France,90.00,POSTAGE,2011-09-01 00:00:00
231226,565246,POST,POSTAGE,4,2011-09-02 10:27:00,15.00,12408,Belgium,60.00,POSTAGE,2011-09-01 00:00:00


# Preparation pipeline

In [334]:
importlib.reload(sys.modules['functions'])
from functions import *

In [335]:
df_train = df_train_ori
df_test = df_test_ori


In [336]:
preparation_pipeline = Pipeline([
    ('features_selector', FeaturesSelector(features_toselect=MODEL_FEATURES)),
    ('bow_encoder', BowEncoder()),
    ('agregate_to_client_level', AgregateToClientLevel(top_value_products))
    
    # Ajouter le log scale du TotalPrice et le MinMaxScale à la fin
    # Faire la réduction dimensionnelle à part pour les bag of words et pour les autres features
    
    #('hour_extractor', HHMM_to_HH()),
    #('data_converter', HHMM_to_Minutes()),
    #('categoricalfeatures_1hotencoder', CategoricalFeatures1HotEncoder()), 
    
    
    #('minmaxscaler', MinMaxScalerMultiple(features_toscale=MODEL_1HOTALL_FEATURES_QUANTITATIVE)),
])

In [337]:
df_train = preparation_pipeline.fit_transform(df_train)

Features selected (in order): Index(['CustomerID', 'DescriptionNormalized', 'InvoiceDate', 'InvoiceMonth',
       'InvoiceNo', 'StockCode', 'TotalPrice'],
      dtype='object')
BowEncoder : Fit data
categorical_features_totransform == ['DescriptionNormalized']
!! categorical_features_totransform['DescriptionNormalized']
Transform data
AgregateToClientLevel : Fit data


In [338]:
df_train

,TotalPricePerMonth,DescriptionNormalized_10,DescriptionNormalized_11,DescriptionNormalized_12,DescriptionNormalized_20,DescriptionNormalized_24,DescriptionNormalized_36,DescriptionNormalized_3d,DescriptionNormalized_50,DescriptionNormalized_60,DescriptionNormalized_72,DescriptionNormalized_acapulco,DescriptionNormalized_acrylic,DescriptionNormalized_airline,DescriptionNormalized_alarm,DescriptionNormalized_alphabet,DescriptionNormalized_am,DescriptionNormalized_and,DescriptionNormalized_angel,DescriptionNormalized_animals,DescriptionNormalized_ant,DescriptionNormalized_antique,DescriptionNormalized_apothecary,DescriptionNormalized_apple,DescriptionNormalized_apples,DescriptionNormalized_apron,DescriptionNormalized_area,DescriptionNormalized_art,DescriptionNormalized_assorted,DescriptionNormalized_asstd,DescriptionNormalized_babushka,DescriptionNormalized_baby,DescriptionNormalized_backpack,DescriptionNormalized_bag,DescriptionNormalized_bakelike,DescriptionNormalized_baking,DescriptionNormalized_ball,DescriptionNormalized_balloon,DescriptionNormalized_balloons,DescriptionNormalized_bank,DescriptionNormalized_baroque,DescriptionNormalized_basket,DescriptionNormalized_bath,DescriptionNormalized_bathroom,DescriptionNormalized_baths,DescriptionNormalized_beaker,DescriptionNormalized_bell,DescriptionNormalized_bells,DescriptionNormalized_bicycle,DescriptionNormalized_bill,DescriptionNormalized_billboard,DescriptionNormalized_bin,DescriptionNormalized_bird,DescriptionNormalized_birdhouse,DescriptionNormalized_birthday,DescriptionNormalized_biscuit,DescriptionNormalized_biscuits,DescriptionNormalized_black,DescriptionNormalized_block,DescriptionNormalized_blocks,DescriptionNormalized_blossom,DescriptionNormalized_blue,DescriptionNormalized_board,DescriptionNormalized_book,DescriptionNormalized_books,DescriptionNormalized_botanical,DescriptionNormalized_bottle,DescriptionNormalized_bowl,DescriptionNormalized_bowls,DescriptionNormalized_box,DescriptionNormalized_boxed,DescriptionNormalized_boxes,DescriptionNormalized_bracelet,DescriptionNormalized_bread,DescriptionNormalized_breakfast,DescriptionNormalized_brown,DescriptionNormalized_brush,DescriptionNormalized_bucket,DescriptionNormalized_buffalo,DescriptionNormalized_building,DescriptionNormalized_bundle,DescriptionNormalized_bunny,DescriptionNormalized_bunting,DescriptionNormalized_butterfly,DescriptionNormalized_cabinet,DescriptionNormalized_cake,DescriptionNormalized_cakes,DescriptionNormalized_cakestand,DescriptionNormalized_calendar,DescriptionNormalized_calm,DescriptionNormalized_can,DescriptionNormalized_candle,DescriptionNormalized_candleholder,DescriptionNormalized_candles,DescriptionNormalized_card,DescriptionNormalized_cards,DescriptionNormalized_carry,DescriptionNormalized_cars,DescriptionNormalized_cases,DescriptionNormalized_cash,...,DescriptionNormalized_stationery,DescriptionNormalized_stencil,DescriptionNormalized_sticker,DescriptionNormalized_stickers,DescriptionNormalized_sticks,DescriptionNormalized_stocking,DescriptionNormalized_storage,DescriptionNormalized_strawberry,DescriptionNormalized_straws,DescriptionNormalized_string,DescriptionNormalized_stripe,DescriptionNormalized_stripes,DescriptionNormalized_stripey,DescriptionNormalized_strongman,DescriptionNormalized_style,DescriptionNormalized_sugar,DescriptionNormalized_suki,DescriptionNormalized_swallows,DescriptionNormalized_sweet,DescriptionNormalized_sweetheart,DescriptionNormalized_swirly,DescriptionNormalized_sympathy,DescriptionNormalized_table,DescriptionNormalized_tag,DescriptionNormalized_tags,DescriptionNormalized_tails,DescriptionNormalized_tall,DescriptionNormalized_tape,DescriptionNormalized_tea,DescriptionNormalized_teacup,DescriptionNormalized_teapot,DescriptionNormalized_teatime,DescriptionNormalized_the,DescriptionNormalized_thermometer,DescriptionNormalized_tidy,DescriptionNormalized_tier,DescriptionNormalized_tile,DescriptionNormalized_time,DescriptionNormalized_tin,DescriptionNormalized_tin

In [339]:
df_train[df_train['BoughtTopValueProduct'] == 1]

,TotalPricePerMonth,DescriptionNormalized_10,DescriptionNormalized_11,DescriptionNormalized_12,DescriptionNormalized_20,DescriptionNormalized_24,DescriptionNormalized_36,DescriptionNormalized_3d,DescriptionNormalized_50,DescriptionNormalized_60,DescriptionNormalized_72,DescriptionNormalized_acapulco,DescriptionNormalized_acrylic,DescriptionNormalized_airline,DescriptionNormalized_alarm,DescriptionNormalized_alphabet,DescriptionNormalized_am,DescriptionNormalized_and,DescriptionNormalized_angel,DescriptionNormalized_animals,DescriptionNormalized_ant,DescriptionNormalized_antique,DescriptionNormalized_apothecary,DescriptionNormalized_apple,DescriptionNormalized_apples,DescriptionNormalized_apron,DescriptionNormalized_area,DescriptionNormalized_art,DescriptionNormalized_assorted,DescriptionNormalized_asstd,DescriptionNormalized_babushka,DescriptionNormalized_baby,DescriptionNormalized_backpack,DescriptionNormalized_bag,DescriptionNormalized_bakelike,DescriptionNormalized_baking,DescriptionNormalized_ball,DescriptionNormalized_balloon,DescriptionNormalized_balloons,DescriptionNormalized_bank,DescriptionNormalized_baroque,DescriptionNormalized_basket,DescriptionNormalized_bath,DescriptionNormalized_bathroom,DescriptionNormalized_baths,DescriptionNormalized_beaker,DescriptionNormalized_bell,DescriptionNormalized_bells,DescriptionNormalized_bicycle,DescriptionNormalized_bill,DescriptionNormalized_billboard,DescriptionNormalized_bin,DescriptionNormalized_bird,DescriptionNormalized_birdhouse,DescriptionNormalized_birthday,DescriptionNormalized_biscuit,DescriptionNormalized_biscuits,DescriptionNormalized_black,DescriptionNormalized_block,DescriptionNormalized_blocks,DescriptionNormalized_blossom,DescriptionNormalized_blue,DescriptionNormalized_board,DescriptionNormalized_book,DescriptionNormalized_books,DescriptionNormalized_botanical,DescriptionNormalized_bottle,DescriptionNormalized_bowl,DescriptionNormalized_bowls,DescriptionNormalized_box,DescriptionNormalized_boxed,DescriptionNormalized_boxes,DescriptionNormalized_bracelet,DescriptionNormalized_bread,DescriptionNormalized_breakfast,DescriptionNormalized_brown,DescriptionNormalized_brush,DescriptionNormalized_bucket,DescriptionNormalized_buffalo,DescriptionNormalized_building,DescriptionNormalized_bundle,DescriptionNormalized_bunny,DescriptionNormalized_bunting,DescriptionNormalized_butterfly,DescriptionNormalized_cabinet,DescriptionNormalized_cake,DescriptionNormalized_cakes,DescriptionNormalized_cakestand,DescriptionNormalized_calendar,DescriptionNormalized_calm,DescriptionNormalized_can,DescriptionNormalized_candle,DescriptionNormalized_candleholder,DescriptionNormalized_candles,DescriptionNormalized_card,DescriptionNormalized_cards,DescriptionNormalized_carry,DescriptionNormalized_cars,DescriptionNormalized_cases,DescriptionNormalized_cash,...,DescriptionNormalized_stationery,DescriptionNormalized_stencil,DescriptionNormalized_sticker,DescriptionNormalized_stickers,DescriptionNormalized_sticks,DescriptionNormalized_stocking,DescriptionNormalized_storage,DescriptionNormalized_strawberry,DescriptionNormalized_straws,DescriptionNormalized_string,DescriptionNormalized_stripe,DescriptionNormalized_stripes,DescriptionNormalized_stripey,DescriptionNormalized_strongman,DescriptionNormalized_style,DescriptionNormalized_sugar,DescriptionNormalized_suki,DescriptionNormalized_swallows,DescriptionNormalized_sweet,DescriptionNormalized_sweetheart,DescriptionNormalized_swirly,DescriptionNormalized_sympathy,DescriptionNormalized_table,DescriptionNormalized_tag,DescriptionNormalized_tags,DescriptionNormalized_tails,DescriptionNormalized_tall,DescriptionNormalized_tape,DescriptionNormalized_tea,DescriptionNormalized_teacup,DescriptionNormalized_teapot,DescriptionNormalized_teatime,DescriptionNormalized_the,DescriptionNormalized_thermometer,DescriptionNormalized_tidy,DescriptionNormalized_tier,DescriptionNormalized_tile,DescriptionNormalized_time,DescriptionNormalized_tin,DescriptionNormalized_tin

In [344]:
for s in df[df['CustomerID'] == '12350']['StockCode'].unique():
    if s in top_value_products:
        print(f'{s} : yes')

POST : yes


In [234]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4332 entries, 12346 to 18287
Columns: 604 entries, TotalPricePerMonth to HasEverCancelled
dtypes: float64(603), int64(1)
memory usage: 20.0+ MB


# Annex

## Display some data

In [407]:
df_nocancel = df[df['InvoiceNo'].str.startswith('C') == False]
df_nocancel.reset_index(inplace=True)

df_gbproduct = df_nocancel[['StockCode', 'TotalPrice']].groupby('StockCode').sum()['TotalPrice']

In [408]:
df_nocancel.head(2)

,index,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalPrice,DescriptionNormalized,InvoiceMonth
0,0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30,WHITE HANGING HEART T-LIGHT HOLDER,2010-12-01 00:00:00
1,1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,WHITE METAL LANTERN,2010-12-01 00:00:00


In [409]:
df_nocancel.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 392732 entries, 0 to 392731
Data columns (total 12 columns):
index                    392732 non-null int64
InvoiceNo                392732 non-null object
StockCode                392732 non-null object
Description              392732 non-null object
Quantity                 392732 non-null int64
InvoiceDate              392732 non-null object
UnitPrice                392732 non-null float64
CustomerID               392732 non-null object
Country                  392732 non-null object
TotalPrice               392732 non-null float64
DescriptionNormalized    392732 non-null object
InvoiceMonth             392732 non-null object
dtypes: float64(2), int64(2), object(8)
memory usage: 36.0+ MB


In [410]:
invoice_dates = pd.to_datetime(df_nocancel["InvoiceDate"], format="%Y-%m-%d ")

In [411]:
invoice_dates = pd.to_datetime(df_nocancel["InvoiceDate"])

In [412]:
np.maximum((pd.to_datetime('2011-12-09 12:50:00') - invoice_dates) / (np.timedelta64(1, "M")), 1)[123456]

7.000987928111688

In [413]:
df_gbcustom_firstorder = df_nocancel[['CustomerID', 'InvoiceDate']].groupby('CustomerID').min()

In [414]:
df_nocancel[['CustomerID', 'InvoiceDate']].groupby('CustomerID').min()['InvoiceDate']

CustomerID
12346    2011-01-18 10:01:00
12347    2010-12-07 14:57:00
12348    2010-12-16 19:09:00
12349    2011-11-21 09:51:00
12350    2011-02-02 16:01:00
                ...         
18280    2011-03-07 09:52:00
18281    2011-06-12 10:53:00
18282    2011-08-05 13:35:00
18283    2011-01-06 14:14:00
18287    2011-05-22 10:39:00
Name: InvoiceDate, Length: 4339, dtype: object

In [45]:
(\
   pd.to_datetime('2011-12-09 12:50:00')\
   - pd.to_datetime(df_nocancel[['CustomerID', 'InvoiceDate']].groupby('CustomerID').min()['InvoiceDate'])
)\
  / (np.timedelta64(1, "M"))

CustomerID
12346    10.681693
12347    12.054845
12348    11.753401
12349     0.595472
12350    10.180656
           ...    
18280     9.104864
18281     5.916549
18282     4.138689
18283    11.070179
18287     6.606821
Name: InvoiceDate, Length: 4339, dtype: float64

In [52]:
# Number of months between first order date and last date of the dataset
series_gbclient_nbmonths = np.maximum(\
(
   (
   pd.to_datetime('2011-12-09 12:50:00')\
   - pd.to_datetime(df_nocancel[['CustomerID', 'InvoiceDate']].groupby('CustomerID').min()['InvoiceDate'])
   )\
    / (np.timedelta64(1, "M"))
), 1)

In [ ]:
df_nocancel[['CustomerID', ]]

In [75]:
df_gbcustom_firstorder

,InvoiceDate
CustomerID,
12346,2011-01-18 10:01:00
12347,2010-12-07 14:57:00
12348,2010-12-16 19:09:00
12349,2011-11-21 09:51:00
12350,2011-02-02 16:01:00
...,...
18280,2011-03-07 09:52:00
18281,2011-06-12 10:53:00
18282,2011-08-05 13:35:00


In [48]:
df_nocancel[df_nocancel['CustomerID'] == '18281'].sort_values(by='InvoiceDate', ascending=True)

,index,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalPrice,DescriptionNormalized,InvoiceMonth
153695,157506,556464,22037,ROBOT BIRTHDAY CARD,12,2011-06-12 10:53:00,0.42,18281,United Kingdom,5.04,ROBOT BIRTHDAY CARD,2011-06-01 00:00:00
153696,157507,556464,22716,CARD CIRCUS PARADE,12,2011-06-12 10:53:00,0.42,18281,United Kingdom,5.04,CARD CIRCUS PARADE,2011-06-01 00:00:00
153697,157508,556464,22028,PENNY FARTHING BIRTHDAY CARD,12,2011-06-12 10:53:00,0.42,18281,United Kingdom,5.04,PENNY FARTHING BIRTHDAY CARD,2011-06-01 00:00:00
153698,157509,556464,23007,SPACEBOY BABY GIFT SET,1,2011-06-12 10:53:00,16.95,18281,United Kingdom,16.95,SPACEBOY BABY GIFT SET,2011-06-01 00:00:00
153699,157510,556464,23008,DOLLY GIRL BABY GIFT SET,1,2011-06-12 10:53:00,16.95,18281,United Kingdom,16.95,DOLLY GIRL BABY GIFT SET,2011-06-01 00:00:00
153700,157511,556464,23209,LUNCH BAG DOILEY PATTERN,10,2011-06-12 10:53:00,1.65,18281,United Kingdom,16.50,LUNCH BAG VINTAGE DOILY,2011-06-01 00:00:00
153701,157512,556464,22467,GUMBALL COAT RACK,6,2011-06-12 10:53:00,2.55,18281,United Kingdom,15.30,GUMBALL COAT RACK,2011-06-01 00:00:00


In [14]:
invoice_dates[2000:2010]

2000   2010-12-02 09:43:00
2001   2010-12-02 09:43:00
2002   2010-12-02 09:43:00
2003   2010-12-02 09:43:00
2004   2010-12-02 09:43:00
2005   2010-12-02 09:43:00
2006   2010-12-02 09:43:00
2007   2010-12-02 09:43:00
2008   2010-12-02 09:43:00
2009   2010-12-02 09:43:00
Name: InvoiceDate, dtype: datetime64[ns]

In [15]:
df_nocancel.loc[2000:2010,'InvoiceDate']

2000    2010-12-02 09:43:00
2001    2010-12-02 09:43:00
2002    2010-12-02 09:43:00
2003    2010-12-02 09:43:00
2004    2010-12-02 09:43:00
2005    2010-12-02 09:43:00
2006    2010-12-02 09:43:00
2007    2010-12-02 09:43:00
2008    2010-12-02 09:43:00
2009    2010-12-02 09:43:00
2010    2010-12-02 09:43:00
Name: InvoiceDate, dtype: object

In [16]:
df_nocancel.loc[100000:100010,'InvoiceMonth']

100000    2011-04-01 00:00:00
100001    2011-04-01 00:00:00
100002    2011-04-01 00:00:00
100003    2011-04-01 00:00:00
100004    2011-04-01 00:00:00
100005    2011-04-01 00:00:00
100006    2011-04-01 00:00:00
100007    2011-04-01 00:00:00
100008    2011-04-01 00:00:00
100009    2011-04-01 00:00:00
100010    2011-04-01 00:00:00
Name: InvoiceMonth, dtype: object

In [17]:
df[df['InvoiceNo'].str.startswith('C') == True]['CustomerID'].unique()

array(['14527', '15311', '17548', ..., '12985', '15951', '16446'],
      dtype=object)

In [361]:
# Product codes that contain chars instead of numbers
df[df['StockCode'].str.isalpha()]['StockCode'].unique()

array(['POST', 'D', 'M', 'PADS', 'DOT', 'CRUK'], dtype=object)

# For debug / test (clean code is in functions.py)

In [189]:
df_train = df_train_ori
df_test = df_test_ori

In [190]:
df_train.head(6)

,index,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalPrice,DescriptionNormalized,InvoiceMonth
0,368341,578255,21559,STRAWBERRY LUNCH BOX WITH CUTLERY,1,2011-11-23 12:58:00,2.55,14675,United Kingdom,2.55,STRAWBERRY LUNCH BOX WITH CUTLERY,2011-11-01 00:00:00
1,231394,565292,22950,36 DOILIES VINTAGE CHRISTMAS,12,2011-09-02 11:54:00,1.45,16102,United Kingdom,17.40,36 DOILIES VINTAGE CHRISTMAS,2011-09-01 00:00:00
2,283540,570249,22079,RIBBON REEL HEARTS DESIGN,10,2011-10-10 08:58:00,1.65,17509,United Kingdom,16.50,RIBBON REEL HEARTS DESIGN,2011-10-01 00:00:00
3,363787,577785,23331,DECORATIVE WICKER HEART SMALL,25,2011-11-21 16:25:00,0.63,16369,United Kingdom,15.75,DECORATIVE WICKER HEART SMALL,2011-11-01 00:00:00
4,398952,581310,21559,STRAWBERRY LUNCH BOX WITH CUTLERY,1,2011-12-08 11:43:00,2.55,16442,United Kingdom,2.55,STRAWBERRY LUNCH BOX WITH CUTLERY,2011-12-01 00:00:00
5,398308,581192,22891,TEA FOR ONE POLKADOT,2,2011-12-07 17:05:00,4.25,15883,United Kingdom,8.50,TEA FOR ONE POLKADOT,2011-12-01 00:00:00


In [191]:
df_train

,index,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalPrice,DescriptionNormalized,InvoiceMonth
0,368341,578255,21559,STRAWBERRY LUNCH BOX WITH CUTLERY,1,2011-11-23 12:58:00,2.55,14675,United Kingdom,2.55,STRAWBERRY LUNCH BOX WITH CUTLERY,2011-11-01 00:00:00
1,231394,565292,22950,36 DOILIES VINTAGE CHRISTMAS,12,2011-09-02 11:54:00,1.45,16102,United Kingdom,17.40,36 DOILIES VINTAGE CHRISTMAS,2011-09-01 00:00:00
2,283540,570249,22079,RIBBON REEL HEARTS DESIGN,10,2011-10-10 08:58:00,1.65,17509,United Kingdom,16.50,RIBBON REEL HEARTS DESIGN,2011-10-01 00:00:00
3,363787,577785,23331,DECORATIVE WICKER HEART SMALL,25,2011-11-21 16:25:00,0.63,16369,United Kingdom,15.75,DECORATIVE WICKER HEART SMALL,2011-11-01 00:00:00
4,398952,581310,21559,STRAWBERRY LUNCH BOX WITH CUTLERY,1,2011-12-08 11:43:00,2.55,16442,United Kingdom,2.55,STRAWBERRY LUNCH BOX WITH CUTLERY,2011-12-01 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...
361438,138706,554019,21714,CITRONELLA CANDLE GARDEN POT,12,2011-05-20 13:16:00,1.25,17591,United Kingdom,15.00,CITRONELLA CANDLE GARDEN POT,2011-05-01 00:00:00
361439,180808,559372,22494,EMERGENCY FIRST AID TIN,12,2011-07-08 10:23:00,1.25,12461,Switzerland,15.00,EMERGENCY FIRST AID TIN,2011-07-01 00:00:00
361440,258582,568063,23351,ROLL WRAP 50'S CHRISTMAS,12,2011-09-23 13:24:00,1.25,15981,United Kingdom,15.00,ROLL WRAP 50'S CHRISTMAS,2011-09-01 00:00:00
361441,190532,560534,21658,GLASS BEURRE DISH,4,2011-07-19 12:10:00,3.95,14911,EIRE,15.80,GLASS BEURRE DISH,2011-07-01 00:00:00


In [192]:
df_train_nocancel = df_train[df_train['InvoiceNo'].str.startswith('C') == False]
df_train_nocancel.reset_index(inplace=True)

In [193]:
feat_list = ['CustomerID', 'TotalPrice']
feat_list_bow = [col for col in df_train_nocancel if col.startswith('DescriptionNormalized_')]
feat_list.extend(feat_list_bow)

In [194]:
feat_list

['CustomerID', 'TotalPrice']

In [195]:
df_train_gbcust_nocancel = df_train_nocancel[feat_list].groupby('CustomerID').sum()

In [196]:
df_train_gbcust_nocancel[feat_list_bow] = df_train_gbcust_nocancel[feat_list_bow].clip(upper=1)

In [197]:
df_train_gbcust_nocancel

,TotalPrice
CustomerID,
12346,77183.60
12347,4055.18
12348,1780.24
12349,1579.51
12350,319.40
...,...
18280,180.60
18281,80.82
18282,160.26


In [198]:
# Number of months between first order date and last date of the dataset
series_train_gbclient_nbmonths = np.maximum(\
(
   (
   pd.to_datetime('2011-12-09 12:50:00')\
   - pd.to_datetime(df_train_nocancel[['CustomerID', 'InvoiceDate']].groupby('CustomerID').min()['InvoiceDate'])
   )\
    / (np.timedelta64(1, "M"))
), 1)

In [199]:
series_train_gbclient_nbmonths

CustomerID
12346    10.681693
12347    12.054845
12348    11.753401
12349     1.000000
12350    10.180656
           ...    
18280     9.104864
18281     5.916549
18282     4.138689
18283    11.070179
18287     6.606821
Name: InvoiceDate, Length: 4332, dtype: float64

In [200]:
df_train_gbcust_nocancel['TotalPrice'] 

CustomerID
12346    77183.60
12347     4055.18
12348     1780.24
12349     1579.51
12350      319.40
           ...   
18280      180.60
18281       80.82
18282      160.26
18283     1832.45
18287     1664.18
Name: TotalPrice, Length: 4332, dtype: float64

In [201]:
df_train_gbcust_nocancel['TotalPrice'] = df_train_gbcust_nocancel['TotalPrice'] / series_train_gbclient_nbmonths

In [202]:
df_train_gbcust_nocancel

,TotalPrice
CustomerID,
12346,7225.783259
12347,336.394209
12348,151.465942
12349,1579.510000
12350,31.373223
...,...
18280,19.835551
18281,13.659991
18282,38.722410


In [203]:
df_train

,index,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalPrice,DescriptionNormalized,InvoiceMonth
0,368341,578255,21559,STRAWBERRY LUNCH BOX WITH CUTLERY,1,2011-11-23 12:58:00,2.55,14675,United Kingdom,2.55,STRAWBERRY LUNCH BOX WITH CUTLERY,2011-11-01 00:00:00
1,231394,565292,22950,36 DOILIES VINTAGE CHRISTMAS,12,2011-09-02 11:54:00,1.45,16102,United Kingdom,17.40,36 DOILIES VINTAGE CHRISTMAS,2011-09-01 00:00:00
2,283540,570249,22079,RIBBON REEL HEARTS DESIGN,10,2011-10-10 08:58:00,1.65,17509,United Kingdom,16.50,RIBBON REEL HEARTS DESIGN,2011-10-01 00:00:00
3,363787,577785,23331,DECORATIVE WICKER HEART SMALL,25,2011-11-21 16:25:00,0.63,16369,United Kingdom,15.75,DECORATIVE WICKER HEART SMALL,2011-11-01 00:00:00
4,398952,581310,21559,STRAWBERRY LUNCH BOX WITH CUTLERY,1,2011-12-08 11:43:00,2.55,16442,United Kingdom,2.55,STRAWBERRY LUNCH BOX WITH CUTLERY,2011-12-01 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...
361438,138706,554019,21714,CITRONELLA CANDLE GARDEN POT,12,2011-05-20 13:16:00,1.25,17591,United Kingdom,15.00,CITRONELLA CANDLE GARDEN POT,2011-05-01 00:00:00
361439,180808,559372,22494,EMERGENCY FIRST AID TIN,12,2011-07-08 10:23:00,1.25,12461,Switzerland,15.00,EMERGENCY FIRST AID TIN,2011-07-01 00:00:00
361440,258582,568063,23351,ROLL WRAP 50'S CHRISTMAS,12,2011-09-23 13:24:00,1.25,15981,United Kingdom,15.00,ROLL WRAP 50'S CHRISTMAS,2011-09-01 00:00:00
361441,190532,560534,21658,GLASS BEURRE DISH,4,2011-07-19 12:10:00,3.95,14911,EIRE,15.80,GLASS BEURRE DISH,2011-07-01 00:00:00


In [204]:
custid_cancelled = df_train[df_train['InvoiceNo'].str.startswith('C') == True]['CustomerID'].unique()